In [465]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain, combinations

In [466]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1751,tt0443680,30000000,14711793,The Assassination of Jesse James by the Coward...,Brad Pitt|Casey Affleck|Sam Shepard|Mary-Louis...,Andrew Dominik,Beyond the myth lies America's greatest betrayal,Outlaw Jesse James is rumored be the 'fastest ...,160,Action|Drama|Western,Plan B Entertainment|Virtual Studios|Scott Fre...,9/2/2007,6.9,2007
81,tt3450650,30000000,107597242,Paul Blart: Mall Cop 2,Kevin James|Raini Rodriguez|Eduardo VerÃ¡stegu...,Andy Fickman,Vegas has a new high roller.,Security guard Paul Blart is headed to Las Veg...,94,Action|Adventure|Comedy|Family,Columbia Pictures|Happy Madison Productions|So...,4/16/2015,5.3,2015
530,tt0271027,25000000,64437847,Kiss of the Dragon,Jet Li|Bridget Fonda|TchÃ©ky Karyo|Max Ryan|Ri...,Chris Nahon,Kiss Fear Goodbye,"Liu Jian, an elite Chinese police officer, com...",98,Action|Crime|Thriller,Twentieth Century Fox Film Corporation|Current...,7/6/2001,6.3,2001
908,tt0277296,60000000,165333180,The Scorpion King,Dwayne Johnson|Kelly Hu|Michael Clarke Duncan|...,Chuck Russell,Warrior. Legend. King.,"In ancient Egypt, peasant Mathayus is hired to...",92,Action|Fantasy|Adventure,Universal Pictures|Kalima Productions GmbH & C...,4/16/2002,5.2,2002
1342,tt0329774,87000000,71073932,xXx: State of the Union,Ice Cube|Willem Dafoe|Samuel L. Jackson|Scott ...,Lee Tamahori,Prepare for the next level,"Ice Cube stars as Darius Stone, a thrill-seeki...",101,Action|Crime|Adventure,Original Film|Columbia Pictures Corporation|Re...,4/27/2005,4.8,2005


In [467]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [468]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#добавляем новый столбец 'profit':
data['profit'] = data['revenue'] - data['budget']


#добавляем новый столбец 'dir_split', где разделяем режиссеров в список:
data['dir_split'] = data['director'].str.split('|')


#добавляем новый столбец 'cast_split', где разделяем актеров в список:
data['cast_split'] = data['cast'].str.split('|')


#добавляем новый столбец 'prod_comp_split', где разделяем студии в список:
data['prod_comp_split'] = data['production_companies'].str.split('|')


# Создаем функцию, которая вытаскивает 1 элемент (месяц) списка, переданного ей:
def month_me(month_list):
    return month_list[0]
    

#добавляем новый столбец 'month', где добавляем месяцы релизов:
data['month'] = data['release_date'].str.split('/').apply(month_me)


# Создаем функцию, которая считает и возвращавет длинну названия фильма в количестве символов:
def len_title(row_title):
    return len(row_title)


# Создаем функцию, которая считает и возвращавет длинну описания фильма в количестве слов:
def len_overview(row_overview):
    return len(row_overview.split(" "))


# Создаем функцию, которая группирует актеров в пары и возвращает список пар:
def pairs(actors):
    for name in range(1,len(actors)+1):
       return list(combinations(actors,2))

In [469]:
data.describe()

,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [470]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [471]:
# тут пишем ваш код для решения данного вопроса:
data[data['budget'] == data['budget'].max()]['original_title']

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [472]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'

In [473]:
data[data['runtime'] == data['runtime'].max()]['original_title']

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [474]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [475]:
data[data['runtime'] == data['runtime'].min()]['original_title']

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [476]:
answers['4'] = '110'

In [477]:
data['runtime'].mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [478]:
answers['5'] = '107'

In [479]:
data.describe()['runtime']['50%']

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [480]:
answers['6'] = 'Avatar (tt0499549)'

In [481]:
data[data['profit'] == data['profit'].max()]['original_title']

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [482]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [483]:
data[data['profit'] == data['profit'].min()]['original_title']

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [484]:
answers['8'] = '1478'

In [485]:
data[data.revenue > data.budget].original_title.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [486]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [487]:
data[data['revenue'] == data[data['release_year'] == 2008].revenue.max()]['original_title']

599    The Dark Knight
Name: original_title, dtype: object

In [488]:
data.query('release_year == 2008').sort_values('profit', ascending=False).head(1)['original_title']

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [489]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [490]:
data.query('2012 <= release_year <= 2014').sort_values('profit').head(1)['original_title']

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [491]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [492]:
answers['11'] = 'Drama'

In [493]:
genre_max = Counter()
for genre in data.genres.str.split('|'):
    genre_max += Counter(genre)
genre_max.most_common(1)

[('Drama', 782)]

In [494]:
max_genre = pd.Series((data.genres.str.split('|').sum())).value_counts().index[0]
max_genre

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [495]:
answers['12'] = 'Drama'

In [496]:
data_profit = data[data['profit'] > 0]
genre_max = Counter()
for genre in data_profit.genres.str.split('|'):
    genre_max += Counter(genre)
genre_max.most_common(1)

[('Drama', 560)]

In [497]:
data_profit = data[data['profit'] > 0]
profit_genres = pd.Series((data_profit.genres.str.split('|').sum())).value_counts()
profit_genres.index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [498]:
answers['13'] = 'Peter Jackson'

In [499]:
dirs_cut_data = data.explode('dir_split')
dir_revenue = dirs_cut_data.groupby(['dir_split']).revenue.sum()
dir_revenue.sort_values(ascending=False).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [500]:
answers['14'] = 'Robert Rodriguez'

In [501]:
dirs_cut_data_temp = dirs_cut_data[dirs_cut_data.genres.str.contains('Action', na=False)]
dirs_cut_data_temp.dir_split.value_counts().index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [502]:
answers['15'] = 'Chris Hemsworth'

In [503]:
cast_cut_data = data.explode('cast_split')
cast_revenue = cast_cut_data[cast_cut_data['release_year'] == 2012].groupby(['cast_split']).revenue.sum()
cast_revenue.sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [504]:
answers['16'] = 'Matt Damon'

In [505]:
high_budget_data = cast_cut_data[cast_cut_data['budget'] > cast_cut_data['budget'].mean()]
high_budget_data['cast_split'].value_counts().index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [506]:
answers['17'] = 'Action'

In [507]:
cast_cut_data_NC = cast_cut_data[cast_cut_data.cast_split.str.contains('Nicolas Cage', na=False)]
pd.Series((cast_cut_data_NC.genres.str.split('|').sum())).value_counts().index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [508]:
answers['18'] = 'K-19: The Widowmaker'

In [509]:
prod_comp_data = data.explode('prod_comp_split')
prod_comp_data_parpic = prod_comp_data[prod_comp_data.prod_comp_split.str.contains('Paramount Pictures', na=False)]
prod_comp_data_parpic[prod_comp_data_parpic['profit'] == prod_comp_data_parpic['profit'].min()]['original_title']

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [510]:
answers['19'] = '2015'

In [511]:
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).index[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [512]:
answers['20'] = '2014'

In [513]:
prod_comp_data_warbros = prod_comp_data[prod_comp_data.prod_comp_split.str.contains('Warner Bros', na=False)]
prod_comp_data_warbros.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).index[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [514]:
answers['21'] = 'Сентябрь'

In [515]:
data.groupby('month')['original_title'].count().sort_values(ascending=False).index[0]

'9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [516]:
answers['22'] = '450'

In [517]:
total_film_amount = data.groupby('month')['original_title'].count()
summer_film_amount = total_film_amount['6'] + total_film_amount['7'] + total_film_amount['8']
summer_film_amount

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [518]:
answers['23'] = 'Peter Jackson'

In [519]:
data_dir_temp = data.explode('dir_split')
data_dir_temp.head()
pivot = data_dir_temp.loc[data_dir_temp['month'].isin(['12','1','2'])].pivot_table(values='original_title',
index='dir_split', 
columns='month', 
aggfunc='count', 
fill_value=0, 
margins=True )
pivot['All'].sort_values(ascending=False).index[1]


'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [520]:
answers['24'] = 'Four By Two Productions'

In [521]:
prod_comp_data['title_len'] = prod_comp_data['original_title'].apply(len_title)
prod_comp_data.groupby(['prod_comp_split'])['title_len'].mean().sort_values(ascending=False).index[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [522]:
answers['25'] = 'Midnight Picture Show'

In [523]:
prod_comp_data['overview_len'] = prod_comp_data['overview'].apply(len_overview)
prod_comp_data.groupby(['prod_comp_split'])['overview_len'].mean().sort_values(ascending=False).index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [524]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [525]:
data.loc[data['vote_average']>data.quantile(0.99, numeric_only=True)['vote_average']]['original_title']

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [526]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint '

In [527]:
data['cast_list'] = data['cast_split'].apply(pairs)
data_pairs = data.explode('cast_list')
data_pairs['cast_list'].value_counts().index[0]

('Daniel Radcliffe', 'Rupert Grint')

# Submission

In [528]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint '}

In [529]:
# и убедиться что ни чего не пропустил)
len(answers)

27